# Create TF Lite for Microcontrollers Model
Run this notebook after executing TF-Logic-Gates-Demo.json.

To convert a TF Model to a TFLM model the following steps are required:
 - Convert TF model to TFLite model
 - Convert TFLite model to TFLM model


For more detail on TensorFlow Lite for Microcontrollers see:
 - https://www.tensorflow.org/lite/microcontrollers
 - https://www.tensorflow.org/lite/models/convert/convert_models
 - https://www.tensorflow.org/lite/models/convert/
 - https://www.tensorflow.org/lite/microcontrollers/build_convert
 - https://www.digikey.com/en/maker/projects/intro-to-tinyml-part-1-training-a-model-for-arduino-in-tensorflow/8f1fc8c0b83d417ab521c48864d2a8ec
 - https://www.digikey.com/en/maker/projects/intro-to-tinyml-part-2-deploying-a-tensorflow-lite-model-to-arduino/59bf2d67256f4b40900a3fa670c14330
 - https://github.com/tensorflow/tflite-micro/blob/main/tensorflow/lite/micro/docs/new_platform_support.md

In [1]:
import tensorflow as tf
from pathlib import Path

All available models:

In [2]:
models = ["and-gate", "nor-gate", "or-gate", "xor-gate", "xnor-gate"]

Configuration

In [3]:
dataset = tf.constant([[0,0], [0,1], [1,0], [1,1]], dtype=tf.float32)
output_dir = Path("TFLM-models")

## Helper functions
### Create representative dataset as signature

In [4]:
def representative_dataset():
  for data in dataset:
    yield {
        "input_1": data,
        "input_2": data,
        "input_3": data,
        "input_4": data,
        "input_5": data,
    }

### Covert Model to TensorFlow Lite Model

In [6]:
def convert_tf_to_tfLite(tf_model_name):
    # Convert the model
    converter = tf.lite.TFLiteConverter.from_saved_model(tf_model_name) # path to the SavedModel directory
    tflite_model = converter.convert()
    
    # Conversion Settings
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.representative_dataset = representative_dataset
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter.inference_input_type = tf.int8  # or tf.uint8
    converter.inference_output_type = tf.int8  # or tf.uint8
    return converter.convert()

Store a TensorFlow Lite model

In [7]:
def store_tfLite_model(tflite_model, model_name):
    with open(model_name + '.tflite', 'wb') as f:
        f.write(tflite_model)

### Translate TensorFlow Lite model to TensorFlow Lite for Microcontrollers Model

Soruce: https://www.digikey.com/en/maker/projects/intro-to-tinyml-part-1-training-a-model-for-arduino-in-tensorflow/8f1fc8c0b83d417ab521c48864d2a8ec

In [8]:
# Function: Convert some hex value into an array for C programming
# soruce: https://www.digikey.com/en/maker/projects/
# intro-to-tinyml-part-1-training-a-model-for-arduino-in-tensorflow/8f1fc8c0b83d417ab521c48864d2a8ec
def hex_to_c_array(hex_data, var_name):
  c_str = ''

  # Create header guard
  c_str += '#ifndef ' + var_name.upper() + '_H\n'
  c_str += '#define ' + var_name.upper() + '_H\n\n'

  # Add array length at top of file
  c_str += '\nunsigned int ' + var_name + '_len = ' + str(len(hex_data)) + ';\n'

  # Declare C variable
  c_str += 'unsigned char ' + var_name + '[] = {'
  hex_array = []
  for i, val in enumerate(hex_data) :

    # Construct string from hex
    hex_str = format(val, '#04x')

    # Add formatting so each line stays within 80 characters
    if (i + 1) < len(hex_data):
      hex_str += ','
    if (i + 1) % 12 == 0:
      hex_str += '\n '
    hex_array.append(hex_str)

  # Add closing brace
  c_str += '\n ' + format(' '.join(hex_array)) + '\n};\n\n'

  # Close out header guard
  c_str += '#endif //' + var_name.upper() + '_H'

  return c_str

In [9]:
def convert_tfLite_to_TFLM(model, model_name):    
    # Create output directory (if not exist)
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    # Write TFLite model to a C source (or header) file
    with open(output_dir / Path(model_name + '.h'), 'w') as file:
      file.write(hex_to_c_array(model, model_name))

## Convert all Models from TF to TFLM

In [10]:
def convert_tf_to_TFLM(tf_model_name):
    tfLite_model = convert_tf_to_tfLite(tf_model_name)
    store_tfLite_model(tfLite_model, tf_model_name)
    convert_tfLite_to_TFLM(tfLite_model, tf_model_name)

In [11]:
for m in models:
    convert_tf_to_TFLM(m)

# Next steps on running TFLM
 - Only working on linux:
     - Clone TFLM git https://github.com/tensorflow/tflite-micro
     - Generate TFLM by following instructions: https://github.com/tensorflow/tflite-micro/blob/main/tensorflow/lite/micro/docs/new_platform_support.md
